In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import h5py
import os
from astropy.wcs import WCS
from reproject import reproject_interp
import numpy as np
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import LogNorm
import astropy.io.fits as fits
from reproject import reproject_from_healpix
from matplotlib import cm
from reproject import reproject_to_healpix
from astropy_healpix import HEALPix
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic
from astropy import units as u
import copy
from mpl_toolkits.axes_grid1 import make_axes_locatable
import gc
from astropy.nddata import Cutout2D

## 1) Mask bad frequencies

In [ ]:
freq_masking = False

In [ ]:
if freq_masking:

    #hduQ = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/Q_400_729_Mar2024_new_gal.fits')
    #hdrQ = hduQ[0].header
    #Q = hduQ[0].data
    
    #hduU = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/U_400_729_Mar2024_new_gal.fits')
    #hdrU = hduU[0].header
    #U = hduU[0].data

    hduV = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/V_400_729_Mar2024_new_gal.fits')
    hdrV = hduV[0].header
    V = hduV[0].data

    # This is based on the October 2023 version, which was stitched together
    # from various subsets of the full band (CHIME_RMsynth_Sep2023.ipynb),
    # with the addition of channel 721, which was found to also be bad.
    # These indices work for the 729 - 400 MHz range. Add 180 to get indices
    # for 400 - 800 MHz, and get additional flags for 729 - 800 MHz.
    
    idx_bad = np.array([4, 6, 25, 67, 68, 76, 78, 82, 83, 91, 92, 95, 96, 115, 139, 
               147, 148, 157, 191, 192, 203, 204, 222, 262, 281, 323, 324,
               332, 334, 433, 442, 447, 448, 516, 518, 526, 527, 528, 529, 
               530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 
               542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 568, 
               569, 570, 571, 572, 595, 596, 597, 598, 599, 600, 601, 620, 
               621, 622, 623, 624, 625, 632, 633, 634, 651, 656, 657, 658, 
               659, 663, 666, 669, 678, 689, 698, 707, 712, 719, 720, 721,
               730, 722, 723, 724, 726, 734, 757, 774, 784, 785, 786, 787, 
               788, 799, 800, 836])

In [ ]:
if freq_masking:

    #Q[idx_bad] = np.nan
    #U[idx_bad] = np.nan
    V[idx_bad] = np.nan
    
    #fits.writeto('/srv/data/chime/chime_IQUV_Mar2024_400_729/Q_400_729_Mar2024_new_gal_fix.fits',
    #             Q,header=hdrQ,overwrite=True,output_verify='fix')
    
    #fits.writeto('/srv/data/chime/chime_IQUV_Mar2024_400_729/U_400_729_Mar2024_new_gal_fix.fits',
    #             U,header=hdrU,overwrite=True,output_verify='fix')

    fits.writeto('/srv/data/chime/chime_IQUV_Mar2024_400_729/V_400_729_Mar2024_new_gal_fix.fits',
                 V,header=hdrV,overwrite=True,output_verify='fix')

    #del Q
    #del U
    del V
    #del hduQ
    #del hduU
    del hduV
    gc.collect()

## 2) Check the Q and U maps

In [ ]:
QU_checks = False

In [ ]:
if QU_checks:

    hduQ_new = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/Q_400_729_Mar2024_new_gal.fits')
    hduU_new = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/U_400_729_Mar2024_new_gal.fits')
    hduQ_old = fits.open('/srv/data/chime/chime_QU_Oct2023_400_729/Q_400_729_Oct2023_new_gal_fix.fits')
    hduU_old = fits.open('/srv/data/chime/chime_QU_Oct2023_400_729/U_400_729_Oct2023_new_gal_fix.fits')
    hdr = hduQ_new[0].header
    freq = WCS(hdr).all_pix2world(0,0,range(hduQ_new[0].data.shape[0]),0)[2]
    print(freq)

In [ ]:
if QU_checks:
    #######################
    idx = 780
    lon_range = [180,90]
    lat_range = [-20,30]
    fs = 12
    vmax = 3
    #######################
    
    crange = SkyCoord(lon_range, lat_range, frame=Galactic, unit=(u.deg, u.deg))
    wcs = WCS(hdr).dropaxis(2).celestial
    
    fig = plt.figure(figsize=(16,12))
    ax1 = fig.add_subplot(321, projection=wcs.celestial)
    ax2 = fig.add_subplot(322, projection=wcs.celestial)
    ax3 = fig.add_subplot(323, projection=wcs.celestial)
    ax4 = fig.add_subplot(324, projection=wcs.celestial)
    ax5 = fig.add_subplot(325, projection=wcs.celestial)
    ax6 = fig.add_subplot(326, projection=wcs.celestial)
    
    im1 = ax1.imshow(hduQ_old[0].data[idx],vmin=-vmax,vmax=vmax,cmap='RdBu_r')
    im2 = ax2.imshow(hduU_old[0].data[idx],vmin=-vmax,vmax=vmax,cmap='RdBu_r')
    im3 = ax3.imshow(hduQ_new[0].data[idx],vmin=-vmax,vmax=vmax,cmap='RdBu_r')
    im4 = ax4.imshow(hduU_new[0].data[idx],vmin=-vmax,vmax=vmax,cmap='RdBu_r')
    im5 = ax5.imshow(hduQ_new[0].data[idx]-hduQ_old[0].data[idx],vmin=-vmax,vmax=vmax,cmap='RdBu_r')
    im6 = ax6.imshow(hduU_new[0].data[idx]-hduU_old[0].data[idx],vmin=-vmax,vmax=vmax,cmap='RdBu_r')
    
    titles = ['Q old','U old','Q new','U new','Q new - old','U new - old']
    axs = [ax1,ax2,ax3,ax4,ax5,ax6]
    ims = [im1,im2,im3,im4,im5,im6]
    
    for i in range(0,len(axs)):
        axs[i].set_ylim(wcs.world_to_pixel(crange)[1])
        axs[i].set_xlim(wcs.world_to_pixel(crange)[0])
        cbar = fig.colorbar(ims[i], ax=axs[i], shrink=1, pad=0.01)
        axs[i].set_title(titles[i]+' '+str(np.round(freq[idx]/1e6,3))+' MHz',fontsize=fs)
        axs[i].set_xlabel(' ')
        axs[i].set_ylabel(' ')

In [ ]:
if QU_checks:
    del hduQ_new
    del hduU_new
    del hduQ_old
    del hduU_old
    gc.collect()

## 3) Make tadpole cutout

In [ ]:
cutout = False

In [ ]:
if cutout:

    #hdu_Q = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/Q_400_729_Mar2024_new_gal_fix.fits')
    #hdu_U = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/U_400_729_Mar2024_new_gal_fix.fits')    
    hdu_V = fits.open('/srv/data/chime/chime_IQUV_Mar2024_400_729/V_400_729_Mar2024_new_gal_fix.fits')
    #Q = hdu_Q[0].data
    #U = hdu_U[0].data
    V = hdu_V[0].data 
    hdr = hdu_V[0].header

    hdr2D = hdr.copy()
    hdr2D['NAXIS'] = 2
    for card in hdr.cards:
        try:
            if card[0][5] == '3':
                del hdr2D[card[0]]
        except:
            pass
        
    print(repr(hdr2D))

In [ ]:
if cutout:
    
    center_spatial = (365,765)  # Replace with your desired coordinates
    size_spatial = 250  # Replace with your desired size
    
    #Q_cut = np.empty([Q.shape[0],size_spatial,size_spatial])
    #U_cut = np.empty([U.shape[0],size_spatial,size_spatial])
    V_cut = np.empty([V.shape[0],size_spatial,size_spatial])
    
    for i in range(0,V.shape[0]):
        print(i)
        #Q_cut[i] = Cutout2D(Q[i], position=center_spatial, size=size_spatial, wcs=WCS(hdr2D)).data
        #U_cut[i] = Cutout2D(U[i], position=center_spatial, size=size_spatial, wcs=WCS(hdr2D)).data
        V_cut[i] = Cutout2D(V[i], position=center_spatial, size=size_spatial, wcs=WCS(hdr2D)).data


In [ ]:
if cutout:

    wcs_new = Cutout2D(V[0], position=center_spatial, size=size_spatial, wcs=WCS(hdr2D))
    hdr_new = wcs_new.wcs.to_header()
    hdr_new['NAXIS'] = hdr['NAXIS']
    hdr_new['WCSAXES'] = 3
    hdr_new['CDELT3'] = hdr['CDELT3']
    hdr_new['CUNIT3'] = hdr['CUNIT3']
    hdr_new['CRVAL3'] = hdr['CRVAL3']
    hdr_new['CRPIX3'] = hdr['CRPIX3']
    hdr_new['CTYPE3'] = hdr['CTYPE3']
    hdr_new['HISTORY'] = 'cut-out for tadpole, 400-729 MHz'
    hdr_new['HISTORY'] = 'made with corrected Stokes U, March 2024'
    
    print(repr(hdr_new))

    #fits.writeto('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/Q_400_729_Mar2024_tadpole_gal.fits',
    #             Q_cut,hdr_new,overwrite=True)
    #fits.writeto('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/U_400_729_Mar2024_tadpole_gal.fits',
    #             U_cut,hdr_new,overwrite=True)
    fits.writeto('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/V_400_729_Mar2024_tadpole_gal.fits',
                 V_cut,hdr_new,overwrite=True)

## 4) RM synthesis, CLEAN and peak fitting

In [ ]:
RMsynth = False
RMclean = False
RMpeaks = False
RMsynth_frac = False
RMclean_frac = False
RMpeaks_frac = False

In [ ]:
if RMsynth:
    !rmsynth3d \
     /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/Q_400_729_Mar2024_tadpole_gal.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/U_400_729_Mar2024_tadpole_gal.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/CHIME_400_729_freq.txt -v -d 0.5 -l 200 -t

    !mv /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/FDF* /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/
    !mv /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/RMSF* /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/

if RMsynth_frac:
    !rmsynth3d \
     /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/QI_400_729_Mar2024_tadpole_gal.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/UI_400_729_Mar2024_tadpole_gal.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/CHIME_400_729_freq.txt -v -d 0.5 -l 200 -t

    !mv /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/FDF* /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/QU_per_I/
    !mv /srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/RMSF* /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/QU_per_I/

In [ ]:
if RMclean:
    !rmclean3d /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/FDF_tot_dirty.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/RMSF_tot.fits -v -c 0.2

if RMclean_frac:
    !rmclean3d /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/QU_per_I/FDF_tot_dirty.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/QU_per_I/RMSF_tot.fits -v -c 0.1

In [ ]:
if RMpeaks:
    !rmtools_peakfitcube \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/FDF_tot_dirty.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/CHIME_400_729_freq.txt \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/dirty_ -v
if RMpeaks_frac:
    !rmtools_peakfitcube \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/QU_per_I/FDF_tot_dirty.fits \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/CHIME_400_729_freq.txt \
     /srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/QU_per_I/dirty_ -v

## 5) Checking maps and spectra

In [ ]:
check_FD = False

In [ ]:
def find_pixels(lon,lat,wcs):

    skycoord = SkyCoord(lon, lat, frame=Galactic, unit=(u.deg, u.deg))
                        
    x, y = wcs.world_to_pixel(skycoord)
    
    return x, y

def make_maps(FD_idx,vmaxPI=5,vmaxFD=15):
    
    fig = plt.figure(figsize=(16,14))
    ax1 = fig.add_subplot(331, projection=wcs.celestial)
    ax2 = fig.add_subplot(332, projection=wcs.celestial)
    ax3 = fig.add_subplot(333, projection=wcs.celestial)
    ax4 = fig.add_subplot(334, projection=wcs.celestial)
    ax5 = fig.add_subplot(335, projection=wcs.celestial)
    ax6 = fig.add_subplot(336, projection=wcs.celestial)
    ax7 = fig.add_subplot(337, projection=wcs.celestial)
    ax8 = fig.add_subplot(338, projection=wcs.celestial)
    ax9 = fig.add_subplot(339, projection=wcs.celestial)
    
    im1 = ax1.imshow(hdu_dirty_old[0].data[FD_idx],vmin=0,vmax=vmaxPI,cmap='viridis')
    im2 = ax2.imshow(hdu_dirty_new[0].data[FD_idx],vmin=0,vmax=vmaxPI,cmap='viridis')
    im3 = ax3.imshow(hdu_dirty_new[0].data[FD_idx]-hdu_dirty_old[0].data[FD_idx],vmin=-vmaxPI,vmax=vmaxPI,cmap='RdBu_r')

    im4 = ax4.imshow(hdu_clean_old[0].data[FD_idx],vmin=0,vmax=vmaxPI,cmap='viridis')
    im5 = ax5.imshow(hdu_clean_new[0].data[FD_idx],vmin=0,vmax=vmaxPI,cmap='viridis')
    im6 = ax6.imshow(hdu_clean_new[0].data[FD_idx]-hdu_clean_old[0].data[FD_idx],vmin=-vmaxPI,vmax=vmaxPI,cmap='RdBu_r')

    im7 = ax7.imshow(hdu_peaks_old[0].data,vmin=-vmaxFD,vmax=vmaxFD,cmap='RdBu_r')
    im8 = ax8.imshow(hdu_peaks_new[0].data,vmin=-vmaxFD,vmax=vmaxFD,cmap='RdBu_r')
    im9 = ax9.imshow(hdu_peaks_new[0].data-hdu_peaks_old[0].data,vmin=-vmaxFD,vmax=vmaxFD,cmap='RdBu_r')

    axs = [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9]
    ims = [im1,im2,im3,im4,im5,im6,im7,im8,im9]
    FD_str = str(FD[FD_idx])
    titles = ['old dirty at FD='+FD_str,'new dirty at FD='+FD_str,'new - old dirty at FD='+FD_str,
              'old clean at FD='+FD_str,'new clean at FD='+FD_str,'new - old clean at FD='+FD_str,
              'old fitted peak FD','new fitted peak FD','new - old fitted peak FD']
    
    for i in range(0,9):
        axs[i].set_xlabel('Galactic longitude')
        axs[i].set_ylabel('Galactic latitude')
        cbar = fig.colorbar(ims[i], ax=axs[i], shrink=0.9, pad=0.01)
        axs[i].set_title(titles[i])

    return

def make_spectra(lon_pt,lat_pt,vmaxFD=15,FD_max=200):
    
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(121, projection=wcs.celestial)
    ax2 = fig.add_subplot(122)
    
    im1 = ax1.imshow(hdu_peaks_new[0].data,vmin=-vmaxFD,vmax=vmaxFD,cmap='RdBu_r')
    ax1.set_xlabel('Galactic longitude')
    ax1.set_ylabel('Galactic latitude')
    cbar = fig.colorbar(im1, ax=ax1, shrink=0.8, pad=0.01)
    x,y = find_pixels(lon_pt,lat_pt,wcs)
    ax1.scatter(x,y,s=30,color='k')

    ax2.plot(FD,hdu_dirty_old[0].data[:,int(np.round(y,0)),int(np.round(x,0))],label='old dirty',color='blue',linestyle='dashed')
    ax2.plot(FD,hdu_dirty_new[0].data[:,int(np.round(y,0)),int(np.round(x,0))],label='new dirty',color='red',linestyle='dashed')
    ax2.plot(FD,hdu_clean_old[0].data[:,int(np.round(y,0)),int(np.round(x,0))],label='old clean')
    ax2.plot(FD,hdu_clean_new[0].data[:,int(np.round(y,0)),int(np.round(x,0))],label='new clean')
    ax2.set_xlim(-FD_max,FD_max)
    ax2.set_ylim(0,2)
    ax2.legend(loc='upper right')
    ax2.set_xlabel('Faraday depth (rad/m^2)')
    ax2.grid()
    ax2.axvline(x=hdu_peaks_old[0].data[int(np.round(y,0)),int(np.round(x,0))],color='C0',linestyle='dotted')
    ax2.axvline(x=hdu_peaks_new[0].data[int(np.round(y,0)),int(np.round(x,0))],color='C1',linestyle='dotted')

    return

In [ ]:
if check_FD:

    hdu_dirty_new = fits.open('/srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/FDF_tot_dirty.fits')
    hdu_clean_new = fits.open('/srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/FDF_clean_tot.fits')
    hdu_peaks_new = fits.open('/srv/data/chime/tadpole_cutout_Mar2024/RMsynth_400_729/dirty_phiPeakPIfit_rm2.fits')
    
    hdu_dirty_old = fits.open('/srv/data/chime/tadpole_cutout/RMsynth_400_729/QU/FDF_tot_dirty.fits')
    hdu_clean_old = fits.open('/srv/data/chime/tadpole_cutout/RMsynth_400_729/QU/clean_200mJy/FDF_clean_tot.fits')
    hdu_peaks_old = fits.open('/srv/data/chime/tadpole_cutout/RMsynth_400_729/QU/dirty_phiPeakPIfit_rm2.fits')
    
    hdr_new = hdu_dirty_new[0].header
    hdr_old = hdu_dirty_old[0].header
    wcs = WCS(hdr_new).celestial
    print(wcs)
    
    FD = WCS(hdr_new).all_pix2world(0,0,range(hdu_dirty_new[0].data.shape[0]),0)[2]


In [ ]:
if check_FD:
    FD_pick = -7
    FD_idx = np.where(abs(FD-FD_pick) == np.nanmin(abs(FD-FD_pick)))[0][0]
    print(FD_idx,FD[FD_idx])
    
    make_maps(FD_idx,vmaxPI=1.3,vmaxFD=12)

    plt.savefig('../PLOTS/fixed_U_compare_FD.png')

In [ ]:
if check_FD:
    make_spectra(137.1,7.1,vmaxFD=12,FD_max=35)

    plt.savefig('../PLOTS/fixed_U_example_spectrum.png')

## 6) Make Q/I and U/I

In [ ]:
make_divI = False

In [ ]:
if make_divI:

    hduQ = fits.open('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/Q_400_729_Mar2024_tadpole_gal.fits')
    hduU = fits.open('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/U_400_729_Mar2024_tadpole_gal.fits')
    hduI = fits.open('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/I_400_729_Oct2023_tadpole_gal.fits')

    fits.writeto('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/QI_400_729_Mar2024_tadpole_gal.fits',
                 hduQ[0].data/hduI[0].data,hduQ[0].header,overwrite=True)
    fits.writeto('/srv/data/chime/tadpole_cutout_Mar2024/IQUV_400_729/UI_400_729_Mar2024_tadpole_gal.fits',
                 hduU[0].data/hduI[0].data,hduU[0].header,overwrite=True)